# Prevención Fuga de Clientes
El enfoque de la prevención es disminuir el impacto latente de la posibilidad de que alguno de los clientes deje de comprar productos. 
Con este fin se realiza la estrategia en 3 sentidos: <br>
**1.Motor de Recomendación** <br>
**Objetivo:** Aumentar la penetración de los productos a los clientes con el fin de fortalecer la relación comercial y su fidelización. <br>
Este enfoque hace parte de la primera parte del notebook, con el objetivo de generar un motor de recomendación con el cual se facilite la venta de nuevos productos por medio de estrategia de datos. <br>
*Notebook 1: Motor de recomendación* <br>

**2.Nueva segmentación de información con base en los datos de los clientes y sus transacciones** <br>
**Objetivo:** Generar nueva información de los clientes, con el objetivo de realizar una mejor gestión de estos y plantear estrategias con base en esta nueva característica de información desarrollada. <br>
El segundo enfoque es a través de una nueva segmentación, que permita de igual forma identificar no solo como y que es posible comprar, sino generar nuevas dimensiones de análisis, para que los equipos de business intelligence, comerciales y financieros, tengan más herramientas durante sus estrategias de negocio. <br>
*Notebook 2: Segmentación de datos* <br>

**3.Predecir la Fuga de clientes** <br>
**Objetivo:** Desarrollar un marcaje de fuga de clientes con base en la historia transaccional de cliente, permitiendo esto entrenar un modelo de Machine Learning para predecir la posible baja de clientes. <br>
El Tercer enfoque tiene dos frentes de trabajo, el primero permite establecer a través de variables de tendencias de compra y montos, si el cliente se encuentra "vivo" o no. Con base en este resultado, se activa el segundo frente, el cual a través de algoritmos de Machine Learning permite predecir la fuga de clientes con base en el marcaje realizado en la primera etapa. <br>
*Notebook 3: Predicción Fuga Clientes* <br>


### Fuente de datos
Los datos usando durante la tesis son de dominio publico y hacen referencia a un comercio de Brasil. <br>
Estos datos han sido modelados en un modelo dimensional con el fin de mejorar el performance del análisis del actual tesis y permitir generar modelos de reporting/dashboard con herramientas de BI.<br>
Los datos han sido migrados a la plataforma github con el fin que sean de dominio publico. <br>


##1. Motor de Recomendación

Durante esta etapa se realizarán dos enfoques de recomendación, el primero con base en productos y el segundo con base en el comportamiento de clientes.


### 1.1 Carga de librerias y datos

In [0]:
#!pip install turicreate
# librerias requeridas
import turicreate as tc
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import time
from sklearn.model_selection import train_test_split


In [0]:
## Localización y carga de los datos
url = 'https://github.com/masdatascience/TFM-AI/blob/master/data_model_completo.xlsx?raw=true'
datos_modelo = pd.ExcelFile(url)
datos_facturacion = pd.read_excel(datos_modelo, sheet_name='fac_txn')
datos_producto = pd.read_excel(datos_modelo, sheet_name='product')
datos_clientes = pd.read_excel(datos_modelo, sheet_name='customer')

# Se elimian los datos nulos de las dimensiones importantes del analisis que son productos y clientes
datos_facturacion = datos_facturacion.dropna(subset=['customer_id'])
datos_facturacion = datos_facturacion.dropna(subset=['product_id'])
# Se elimina posibles chargeback que se presenten
datos_facturacion = datos_facturacion.loc[datos_facturacion['total_value'] > 0]

print("Facturación:"+ str(datos_facturacion.shape) + " Clientes:"+str(datos_clientes.shape)+" Productos:"+str(datos_producto.shape))

### 1.2 Preparación de los datos 
En esta sección se estarán preparando tres enfoques de matrices con el fin de permitir desarrollar los motores de recomendación. La primera es la relación entre el cliente y los productos, la siguiente es una matriz normalizada y la última integra valores dummies para la comparación . 

#### 1.2.1 Relación Cliente - Producto



In [0]:
# Obtengo la Matriz Relación Cliente - Producto , y la normalizo
matrix_usuario_producto =pd.crosstab(datos_facturacion['customer_id'],datos_facturacion['product_id'],values=datos_facturacion['total_value'], aggfunc='sum', margins=True, margins_name="total_value")
matrix_usuario_producto_norm = (matrix_usuario_producto-matrix_usuario_producto.min())/(matrix_usuario_producto.max()-matrix_usuario_producto.min())
d = matrix_usuario_producto_norm.reset_index() 
d.index.names = ['total_frecuencia'] 
data_norm = pd.melt(d, id_vars=['customer_id'], value_name='total_frecuencia').dropna()
print("Matriz normalizada:" +str(data_norm.shape))

Matriz normalizada:(14312, 3)


#### 1.2.2 Relación Cliente Producto - Contraste

In [0]:
datos_dummy = datos_facturacion.copy()
datos_dummy['total_dummy'] = 1
print("Matriz Dummy:"+ str(datos_dummy.shape))

Matriz Dummy:(10000, 22)


#### 1.2.3 Se generan datos de entrenamiento y pruebas

In [0]:
train_data, test_data  = train_test_split(data_norm, test_size=0.3 ,random_state=1)
train_data_dummy, test_data_dummy = train_test_split(datos_dummy, test_size=0.3 ,random_state=1)

#### 1.2.4  Se define función para ejecutar los tipos de motores de recomendación

In [0]:
# Se define una función que retorne un modelo resultado segun el enfoque que se requiere:
# 1. Popularity : Es el item que más se a consumido en todos los clientes
# 2. cosine :
# 3. Pearson: 
def modelo_requerido(modelo,datos ,campo ):

    train_data=tc.SFrame(datos)
    print("Modelo solicitado:"+ str(modelo))
    if modelo == 'producto':
        return  tc.popularity_recommender.create(train_data,user_id='customer_id', item_id='product_id', target=campo)
    elif modelo == 'distancia_coseno':
        return tc.item_similarity_recommender.create(train_data,user_id='customer_id', item_id='product_id', target=campo,similarity_type='cosine')
    elif modelo == 'pearson':
        return tc.item_similarity_recommender.create(train_data,user_id='customer_id', item_id='product_id', target=campo,similarity_type='pearson')
    else : 
        print("ERROR: Modelo solicitado no existe")

#Obtiene las recomendaciones de forma visual para los clientes
def productos_recomendado(modelo, usuarios, cantidad_recomendaciones):
     modelo.recommend(users=list(usuarios), k=cantidad_recomendaciones).print_rows(cantidad_recomendaciones)

#Evalua los modelos generados 
def evaluacion_modelos(modelo, test):
  
  if modelo == 'normalizado':
        modelos_generados= [fac_producto, fac_coseno, fac_pearson]
        nombres_modelos = ['Modelo basado en contenido', 'Modelo Colaborativo - Distancia Coseno', 'Modelo Colaborativo - Pearson']
        test_data= tc.SFrame(test) 
        eval_counts = tc.recommender.util.compare_models(test_data, modelos_generados, model_names=nombres_modelos)
  elif modelo == 'dummy':
        modelos_generados= [dum_producto, dum_coseno, dum_pearson]
        nombres_modelos = ['Modelo Dummy basado en contenido', 'Modelo Dummy Colaborativo - Distancia Coseno', 'Modelo Dummy Colaborativo - Pearson']
        test_data= tc.SFrame(test) 
        eval_counts = tc.recommender.util.compare_models(test_data, modelos_generados, model_names=nombres_modelos)
  else : 
        print("ERROR: Modelo solicitado no existe")
      
#Exporta cinco  recomendaciones por cliente en google Drive      
def exportarRecomendaciones(modelo, clientes):

  lista_clientes =  list(clientes)
  recomendaciones = modelo.recommend(users=lista_clientes, k=5)
  resultado = recomendaciones.to_dataframe()
  resultado['id_producto_recomendado'] = resultado.groupby(['customer_id'])['product_id'].transform(lambda x: ';'.join(x.astype(str)))
  exportar = resultado[['customer_id', 'id_producto_recomendado']].drop_duplicates().sort_values('customer_id').set_index('customer_id')
  exportar.to_csv('recomendaciones.csv')
  !cp recomendaciones.csv drive/My\ Drive/TFM-AI/
  print("Se han exportado las recomendaciones en Google drive")


### 1.3 Modelo de recomendación basado en Contenido
Este modelo genera recomendaciones con base en la relación de los clientes y sus productos

In [0]:
# Se generan las recomendaciones por dataset
fac_producto = modelo_requerido('producto',train_data,'total_frecuencia' )
dum_producto = modelo_requerido('producto' ,train_data_dummy, 'total_dummy')

Modelo solicitado:producto


Preparing data set.

Data has 10018 observations with 6975 users and 1229 items.

Data prepared in: 0.027246s

10018 observations to process; with 1229 unique items.

...

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 7000 observations with 6296 users and 4627 items.

Data prepared in: 0.023713s

7000 observations to process; with 4627 unique items.

In [0]:
# Se generan las recomendaciones para un Cliente
productos_recomendado(fac_producto,'e4bb330e4e0967e8cdd333d1e8b1df2f',5)
productos_recomendado(dum_producto,'e4bb330e4e0967e8cdd333d1e8b1df2f',5)

+-------------+-------------------------------+-------+------+
| customer_id |           product_id          | score | rank |
+-------------+-------------------------------+-------+------+
|      e      | 382132cf96c88a277a65226833... |  1.0  |  1   |
|      e      | 672a21bf17c63f71edbe01ecb9... |  1.0  |  2   |
|      e      | 82a61259a621866c4ba63743da... |  1.0  |  3   |
|      e      | 2e64d11879c5bbda600381dbb5... |  1.0  |  4   |
|      e      | c1f5307decb89342351bec5366... |  1.0  |  5   |
+-------------+-------------------------------+-------+------+
[160 rows x 4 columns]

+-------------+-------------------------------+-------+------+
| customer_id |           product_id          | score | rank |
+-------------+-------------------------------+-------+------+
|      e      | aadff88486740e0b0ebe2be6c0... |  1.0  |  1   |
|      e      | 7c55ea4aea1acf1ce11440010f... |  1.0  |  2   |
|      e      | 68ad45d48d69404aeb71ce87e1... |  1.0  |  3   |
|      e      | 4520766ec412348

### 1.4 Motor de recomendación Colaborativo
Este modelo genera recomendaciones con base en el comportamiento de compra que tienen los usuarios. <br>
Con este fin se realizan pruebas con dos enfoques, la distancia coseno y pearson para evaluar cual es el que da mejores resultados

In [0]:
# Se generan las recomendaciones por dataset
fac_coseno = modelo_requerido('distancia_coseno',train_data,'total_frecuencia' )
dum_coseno = modelo_requerido('distancia_coseno' ,train_data_dummy, 'total_dummy')

Modelo solicitado:distancia_coseno


Preparing data set.

Data has 10018 observations with 6975 users and 1229 items.

Data prepared in: 0.029116s

Training model from provided data.

...

| 138.597ms                           | 100              | 4627            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.143237s

In [0]:
productos_recomendado(fac_coseno,'e4bb330e4e0967e8cdd333d1e8b1df2f',5)
productos_recomendado(dum_coseno,'e4bb330e4e0967e8cdd333d1e8b1df2f',5)

+-------------+-------------------------------+---------------------+------+
| customer_id |           product_id          |        score        | rank |
+-------------+-------------------------------+---------------------+------+
|      e      | a5f6ba3fe91a2e73f10b612131... | 0.07471107482910157 |  1   |
|      e      | 72ef87b1953a2844ed089bd6dc... | 0.07471107482910157 |  2   |
|      e      | 0dadef6ed70b861b14eddb3a3c... | 0.07471107482910157 |  3   |
|      e      | 1b0e39ec889889ea1d492603d8... | 0.07471107482910157 |  4   |
|      e      | 78a727ba961fe7561d533e153b... | 0.07470573544502258 |  5   |
+-------------+-------------------------------+---------------------+------+
[160 rows x 4 columns]

+-------------+-------------------------------+-------+------+
| customer_id |           product_id          | score | rank |
+-------------+-------------------------------+-------+------+
|      e      | aadff88486740e0b0ebe2be6c0... |  0.0  |  1   |
|      e      | 7c55ea4aea1acf1

In [0]:
# Se generan las recomendaciones por dataset
fac_pearson= modelo_requerido('pearson',train_data,'total_frecuencia' )
dum_pearson = modelo_requerido('pearson' ,train_data_dummy, 'total_dummy')

Modelo solicitado:pearson


Preparing data set.

Data has 10018 observations with 6975 users and 1229 items.

Data prepared in: 0.029826s

Training model from provided data.

...

| 136.961ms                           | 100              | 4627            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.141151s

In [0]:
productos_recomendado(fac_pearson,'e4bb330e4e0967e8cdd333d1e8b1df2f',5)
productos_recomendado(dum_pearson,'e4bb330e4e0967e8cdd333d1e8b1df2f',5)

+-------------+-------------------------------+-------+------+
| customer_id |           product_id          | score | rank |
+-------------+-------------------------------+-------+------+
|      e      | 382132cf96c88a277a65226833... |  1.0  |  1   |
|      e      | 672a21bf17c63f71edbe01ecb9... |  1.0  |  2   |
|      e      | 82a61259a621866c4ba63743da... |  1.0  |  3   |
|      e      | 2e64d11879c5bbda600381dbb5... |  1.0  |  4   |
|      e      | c1f5307decb89342351bec5366... |  1.0  |  5   |
+-------------+-------------------------------+-------+------+
[160 rows x 4 columns]

+-------------+-------------------------------+-------+------+
| customer_id |           product_id          | score | rank |
+-------------+-------------------------------+-------+------+
|      e      | aadff88486740e0b0ebe2be6c0... |  0.0  |  1   |
|      e      | 7c55ea4aea1acf1ce11440010f... |  0.0  |  2   |
|      e      | 68ad45d48d69404aeb71ce87e1... |  0.0  |  3   |
|      e      | 4520766ec412348

### 1.5 Evaluación de Modelos 
Con el fin de establecer cual es el mejor modelo entre los tres probados, se realiza una validación de estos de forma conjunta 

In [0]:
evaluacion_modelos('normalizado',test_data)

PROGRESS: Evaluate model Modelo basado en contenido


recommendations finished on 1000/3540 queries. users per second: 80076.9

recommendations finished on 2000/3540 queries. users per second: 77178.4

recommendations finished on 3000/3540 queries. users per second: 76151.8


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    | 0.0008474576271186458 | 0.0005657435707449598 |
|   2    | 0.0007062146892655379 | 0.0008490012657837045 |
|   3    | 0.0007532956685499077 | 0.0014147448365286626 |
|   4    | 0.0007768361581920927 |  0.001980488407273632 |
|   5    | 0.0008474576271186477 | 0.0026874749158716927 |
|   6    | 0.0008474576271186435 | 0.0029707326109104444 |
|   7    | 0.0008474576271186458 |  0.003536476181655398 |
|   8    |  0.00077683615819209  |  0.003537248000987936 |
|   9    | 0.0008160703075957321 |  0.004102991571732884 |
|   10   | 0.0008192090395480234 |  0.004527492204624739 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.35368532108225

Per User RMSE (best)
+-------------------------------+------+------

...


Precision and recall summary statistics by cutoff
+--------+-----------------------+----------------------+
| cutoff |     mean_precision    |     mean_recall      |
+--------+-----------------------+----------------------+
|   1    |  0.029096045197740155 | 0.02881433114136646  |
|   2    |  0.015536723163841828 | 0.03051001821493618  |
|   3    |  0.010734463276836153 |  0.0312170047235342  |
|   4    |  0.008121468926553656 | 0.031217776542866903 |
|   5    |  0.006723163841807925 | 0.03192476305146492  |
|   6    |  0.005649717514124294 | 0.03192553487079736  |
|   7    |  0.004923325262308325 | 0.03206754962798315  |
|   8    |  0.004343220338983058 | 0.032068321447315674 |
|   9    |  0.003954802259886982 | 0.032634065018060404 |
|   10   | 0.0036158192090395623 | 0.032917322713099266 |
+--------+-----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.29733332331030304

Per User RMSE (best)
+-------------------------------+------+-------+
|       

recommendations finished on 1000/3540 queries. users per second: 56650.8

recommendations finished on 2000/3540 queries. users per second: 52987.2

recommendations finished on 3000/3540 queries. users per second: 52453.1


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    | 0.0008474576271186461 | 0.0005657435707449604 |
|   2    |  0.000706214689265538 | 0.0008490012657837039 |
|   3    |  0.000753295668549908 | 0.0014147448365286633 |
|   4    | 0.0007768361581920934 |  0.001980488407273632 |
|   5    | 0.0008474576271186472 | 0.0026874749158716914 |
|   6    | 0.0008474576271186445 | 0.0029707326109104427 |
|   7    | 0.0008474576271186474 |  0.003536476181655398 |
|   8    | 0.0007768361581920886 | 0.0035372480009879387 |
|   9    | 0.0008160703075957321 |  0.004102991571732876 |
|   10   | 0.0008192090395480232 |  0.004527492204624734 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.3616483307276593

Per User RMSE (best)
+-------------------------------+------+----

In [0]:
evaluacion_modelos('dummy',test_data_dummy)  

PROGRESS: Evaluate model Modelo Dummy basado en contenido


recommendations finished on 1000/2869 queries. users per second: 47551.1

recommendations finished on 2000/2869 queries. users per second: 45049.1


Precision and recall summary statistics by cutoff
+--------+------------------------+-----------------------+
| cutoff |     mean_precision     |      mean_recall      |
+--------+------------------------+-----------------------+
|   1    |          0.0           |          0.0          |
|   2    |          0.0           |          0.0          |
|   3    | 0.00011618450098756866 | 0.0003485535029627069 |
|   4    | 8.713837574067672e-05  | 0.0003485535029627069 |
|   5    | 6.971070059254129e-05  | 0.0003485535029627069 |
|   6    | 5.809225049378433e-05  | 0.0003485535029627069 |
|   7    | 9.958671513220173e-05  | 0.0006971070059254119 |
|   8    | 0.00013070756361101477 | 0.0010456605088881182 |
|   9    | 0.00011618450098756836 | 0.0010456605088881182 |
|   10   | 0.00010456605088881173 | 0.0010456605088881177 |
+--------+------------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.0

Per User RMSE (best)
+-------------------------------+------+-----

recommendations finished on 1000/2869 queries. users per second: 39217.2

recommendations finished on 2000/2869 queries. users per second: 37234.9


Precision and recall summary statistics by cutoff
+--------+------------------------+-----------------------+
| cutoff |     mean_precision     |      mean_recall      |
+--------+------------------------+-----------------------+
|   1    | 0.0013942140118508254  | 0.0013942140118508254 |
|   2    | 0.0006971070059254127  | 0.0013942140118508254 |
|   3    | 0.0005809225049378404  |  0.001742767514813529 |
|   4    | 0.00043569187870338223 |  0.001742767514813529 |
|   5    | 0.00034855350296270575 | 0.0017427675148135294 |
|   6    | 0.0002904612524689202  | 0.0017427675148135294 |
|   7    | 0.0002987601453966049  | 0.0020913210177762346 |
|   8    | 0.00030498431509236757 | 0.0024398745207389405 |
|   9    | 0.00027109716897099245 | 0.0024398745207389405 |
|   10   | 0.00024398745207389444 | 0.0024398745207389392 |
+--------+------------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.9960470182500112

Per User RMSE (best)
+-----------------------------

recommendations finished on 1000/2869 queries. users per second: 33644

recommendations finished on 2000/2869 queries. users per second: 32144


Precision and recall summary statistics by cutoff
+--------+------------------------+-----------------------+
| cutoff |     mean_precision     |      mean_recall      |
+--------+------------------------+-----------------------+
|   1    |          0.0           |          0.0          |
|   2    |          0.0           |          0.0          |
|   3    | 0.00011618450098756831 | 0.0003485535029627051 |
|   4    | 8.713837574067628e-05  | 0.0003485535029627051 |
|   5    |  6.97107005925411e-05  | 0.0003485535029627051 |
|   6    | 5.8092250493784154e-05 | 0.0003485535029627051 |
|   7    | 9.958671513220154e-05  | 0.0006971070059254107 |
|   8    | 0.0001307075636110146  | 0.0010456605088881169 |
|   9    | 0.00011618450098756839 | 0.0010456605088881169 |
|   10   | 0.00010456605088881179 | 0.0010456605088881169 |
+--------+------------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0

Per User RMSE (best)
+-------------------------------+------+-----

### 1.6 Exportar recomendaciones
Con los resultado obtenidos podemos identificar que el mejor modelo es el Pearson, por tal motivo se exporta su información . 

In [0]:
exportarRecomendaciones(fac_pearson,datos_clientes.customer_id)

recommendations finished on 1000/96352 queries. users per second: 65299.7

recommendations finished on 2000/96352 queries. users per second: 64576.5

recommendations finished on 3000/96352 queries. users per second: 64448.2

recommendations finished on 4000/96352 queries. users per second: 64197.2

recommendations finished on 5000/96352 queries. users per second: 63987.7

...

recommendations finished on 93000/96352 queries. users per second: 61679.5

recommendations finished on 94000/96352 queries. users per second: 61410.9

recommendations finished on 95000/96352 queries. users per second: 61159.9

recommendations finished on 96000/96352 queries. users per second: 60870.2

Se han exportado las recomendaciones en Google drive
